# Churn Prediction Model Deployment

In the previous session we trained a model for predicting churn and evaluated it. Now let's deploy it.

In [1]:
# Necessary import
import pickle
import requests
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
# read the data
df = pd.read_csv('data-week-3.csv')

# Normalize columns names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# List of categorical variaables
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

# For each categorical variable
for c in categorical_columns:
    # Normalize strings values
    df[c] = df[c].str.lower().str.replace(' ', '_')

# Convert `object` type to numeric
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce')
# Fill missing values
df.totalcharges = df.totalcharges.fillna(0)

# Target encoding
df.churn = (df.churn == 'yes').astype(int)

In [3]:
# Split the data into full_train and test
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

In [4]:
# List of numerical features
numerical = ['tenure', 'monthlycharges', 'totalcharges']

# List of categorical features
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [5]:
# Function for training a logistic regression model
def train(df_train, y_train, C = 1.0):
    # Convert data to list of dictionaries
    dicts = df_train[categorical + numerical].to_dict(orient = 'records')

    # Initialize vectorizer for One-Hot-Encoding
    dv = DictVectorizer(sparse = False)
    # One-Hot-Encoding
    X_train = dv.fit_transform(dicts)

    # Initialize logistic regression model
    model = LogisticRegression(C = C, max_iter = 1000)
    # Model training
    model.fit(X_train, y_train)

    # return one-hot-encoder and logistic model
    return dv, model

In [6]:
def predict(df, dv, model):
     # Convert data to list of dictionaries
    dicts = df[categorical + numerical].to_dict(orient = 'records')

    # One-Hot-Encoding
    X = dv.transform(dicts)
    # Make soft predictions
    y_pred = model.predict_proba(X)[:, 1]

    # return predictions
    return y_pred

In [7]:
# Regularization parameter
C = 1.0
# Number of splits for Kfold Cross-Validation
n_splits = 5

In [8]:
# Kfold cross-validation
kfold = KFold(n_splits = n_splits, shuffle = True, random_state = 1)

# Initialize scores
scores = []

# For each iteration of K-fold split and the pair of indexes generated
for train_idx, val_idx in kfold.split(df_full_train):
    # Select train and validation data
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    # Select target variables
    y_train = df_train.churn.values
    y_val = df_val.churn.values

    # Train model
    dv, model = train(df_train, y_train, C = C)
    # Make predictions
    y_pred = predict(df_val, dv, model)

    # Get score
    auc = roc_auc_score(y_val, y_pred)
    # Store score
    scores.append(auc)
    
# Print scores' means and standard deviations
print('C = %s, mean = %.3f, std = +- %.3f' % (C, np.mean(scores), np.std(scores)))

C:\Users\dell\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dell\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

C = 1.0, mean = 0.842, std = +- 0.007


C:\Users\dell\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# output scores
scores

[0.8444905841071975,
 0.845138475036419,
 0.8333012241621514,
 0.8347529097653003,
 0.8516990367418656]

In [10]:
# Model training
dv, model = train(df_full_train, df_full_train.churn.values, C = 1.0)
# Make soft predictions
y_pred = predict(df_test, dv, model)

# Real test target values
y_test = df_test.churn.values
# AUC score
auc = roc_auc_score(y_test, y_pred)
auc

C:\Users\dell\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.858359875199064

## Save the model

In [19]:
# model file name
output_file = f'model_C={C}.bin'
output_file

'model_C=1.0.bin'

In [20]:
# Create a file to write bytes into it
f_out = open(output_file, 'wb') 
# Save model
pickle.dump((dv, model), f_out)
# Close file
f_out.close()

We can now list available binary files to make sure the model was saved.

In [22]:
ls -lh *.bin

 Le volume dans le lecteur C n’a pas de nom.
 Le numéro de série du volume est 2EFA-F61A

 Répertoire de C:\Users\dell\Videos\Elearning\Data Science\ML_Zoomcamp\ML_Zoomcamp_2024\05_deployment\code


 Répertoire de C:\Users\dell\Videos\Elearning\Data Science\ML_Zoomcamp\ML_Zoomcamp_2024\05_deployment\code

23/10/2024  19:59             2 421 model_C=1.0.bin
               1 fichier(s)            2 421 octets
               0 Rép(s)  359 390 584 832 octets libres


Or we can simply save the model as follows:

In [24]:
# Open file and write into it
with open(output_file, 'wb') as f_out: 
    # Save model
    pickle.dump((dv, model), f_out)

The `with` statement ensures that the file containing the model is automatically closed.

## Load the model

We can first restart, the kernel to make sure the file was saved and we can use it.

Note that we need to reload the library `pickle`.

In [1]:
# necessary import
import pickle

In [3]:
# Name of the model
input_file = 'model_C=1.0.bin'

In [4]:
# Open file to read it
with open(input_file, 'rb') as f_in:
    # Load the model
    dv, model = pickle.load(f_in)

In [6]:
# output the model
model

LogisticRegression(max_iter=1000)

In [9]:
# New customer information
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [11]:
# One-Hot-Encoding
X = dv.transform([customer])

# Make predictions
y_pred = model.predict_proba(X)[0, 1]

In [13]:
# Print customer info and the model's prediction
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6286411785777591


## Making requests

In [41]:
# url address for predictions
url = 'http://localhost:9696/predict'

In [42]:
# New customer information
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [ ]:
# Send a request for model prediction
response = requests.post(url, json = customer).json()

In [39]:
# request response
response

{'churn': True, 'churn_probability': 0.5133820686195286}

In [27]:
# Send a promotional email if necessary
if response['churn']:
    print('sending email to', 'asdx-123d')

sending email to asdx-123d


---